In [1]:
import os

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print
import dask.config
import distributed

# scipy basics
import numpy as np
import rasterio
import rasterio.features
import rasterio.transform
import rasterio.windows

from numba import jit
import concurrent.futures

import boto3
import time
import math
import ctypes
import pandas as pd

<font size="6">Making cloud and local clusters</font> 

In [ ]:
# Full cluster
coiled_cluster = coiled.Cluster(
    n_workers=20,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "32GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

In [2]:
# Test cluster
coiled_cluster = coiled.Cluster(
    n_workers=1,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "64GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

Output()

Package - bcrypt, bcrypt==4.1.0 has no install candidate for linux-64 on conda-forge

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│                                  ╷                                                               │
│   Package                        │ Note                                                          │
│ ╶────────────────────────────────┼─────────────────────────────────────────────────────────────╴ │
│   et-xmlfile                     │ https://pypi.org/pypi                                         │
│                                  ╵                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────── Not Synced with Cluster ─────────────────────────────────────╮
│            ╷                                                                         ╷           │
│   Package  │ Error                                                                   │ Risk      │
│ ╶──────────┼─────────────────────────────────────────────────────────────────────────┼─────────╴ │
│   bcrypt   │ bcrypt==4.1.0 has no install candidate for linux-64 on conda-forge      │ Warning   │
│            ╵                                                                         ╵           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Connection method: Cluster object,Cluster type: coiled.Cluster
Dashboard: https://cluster-rqyqq.dask.host/sB2PALMV8HuXtPI1/status,
Dashboard: https://cluster-rqyqq.dask.host/sB2PALMV8HuXtPI1/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tls://10.0.35.1:8786,Workers: 0
Dashboard: http://10.0.35.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [ ]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client(processes=False)
local_client

In [ ]:
local_client = Client()
local_client

In [ ]:
# Local cluster with multiple workers
local_cluster = LocalCluster()  
local_client = Client(local_cluster)
local_client

<font size="6">Shutting down cloud and local clusters</font> 

In [ ]:
coiled_cluster.shutdown()

In [ ]:
local_client.shutdown()

<font size="6">Analysis</font> 

<font size="4">Paths and functions</font>

In [3]:
# General paths and constants

general_uri = 's3://gfw2-data/landcover/composite/'

s3_out_dir = 'climate/AFOLU_flux_model/LULUCF/outputs'

IPCC_class_max_val = 6

# IPCC codes
forest = 1
cropland = 2
settlement = 3
wetland = 4
grassland = 5
otherland = 6

In [4]:
def timestr():
    return time.strftime("%Y%m%d_%H_%M_%S")

def boundstr(bounds):
    bounds_str = "_".join([str(round(x)) for x in bounds])
    return bounds_str

def calc_chunk_length_pixels(bounds):
    chunk_length_pixels = int((bounds[3]-bounds[1]) * (40000/10))
    return chunk_length_pixels

In [5]:
# Returns list of all chunk boundaries within a bounding box for chunks of a given size
def get_chunk_bounds(chunk_params):

    min_x = chunk_params[0]
    min_y = chunk_params[1]
    max_x = chunk_params[2]
    max_y = chunk_params[3]
    chunk_size = chunk_params[4]
    
    x, y = (min_x, min_y)
    chunks = []

    # Polygon Size
    while y < max_y:
        while x < max_x:
            bounds = [
                x,
                y,
                x + chunk_size,
                y + chunk_size,
            ]
            chunks.append(bounds)
            x += chunk_size
        x = min_x
        y += chunk_size

    return chunks

# Returns the encompassing tile_id string in the form YYN/S_XXXE/W based on a coordinate
def xy_to_tile_id(top_left_x, top_left_y):

    lat_ceil = math.ceil(top_left_y/10.0) * 10
    lng_floor = math.floor(top_left_x/10.0) * 10
    
    lng: str = f"{str(lng_floor).zfill(3)}E" if (lng_floor >= 0) else f"{str(-lng_floor).zfill(3)}W"
    lat: str = f"{str(lat_ceil).zfill(2)}N" if (lat_ceil >= 0) else f"{str(-lat_ceil).zfill(2)}S"

    return f"{lat}_{lng}"

In [6]:
# Lazily opens tile within provided bounds (i.e. one chunk) and returns as a numpy array
# If it can't open the chunk (no data in it), it returns an array of all 0s
def get_tile_dataset_rio(uri, bounds, chunk_length):

    try:
        with rasterio.open(uri) as ds:
            window = rasterio.windows.from_bounds(*bounds, ds.transform)
            data = ds.read(1, window=window)
    except:
        data = np.zeros((chunk_length, chunk_length))

    if data.size==0:
        # dask_print("No data in chunk")
        return np.zeros((chunk_length, chunk_length))
    else:
        # dask_print("Data in chunk")
        return data

In [ ]:
# Prepares list of chunks to download.
# Chunks are defined by a bounding box.
def prepare_to_download_chunk(bounds, start_year, download_dict):
 
    futures = {}

    bounds_str = boundstr(bounds)
    tile_id = xy_to_tile_id(bounds[0], bounds[3])
    chunk_length_pixels = calc_chunk_length_pixels(bounds)

    # Submit requests to S3 for input chunks but don't actually download them yet. This queueing of the requests before downloading them speeds up the downloading
    # Approach is to download all the input chunks up front for every year to make downloading more efficient, even though it means storing more upfront
    with concurrent.futures.ThreadPoolExecutor() as executor:
        
        dask_print(f"Requesting data in chunk {bounds_str} in {tile_id}: {timestr()}")

        for key, value in download_dict.items():
            futures[executor.submit(get_tile_dataset_rio, value, bounds, chunk_length_pixels)] = key

    return futures

In [7]:
def save_and_upload(bounds, chunk_length_pixels, tile_id, bounds_str, year, output_dict):

    transform = rasterio.transform.from_bounds(*bounds, width=chunk_length_pixels, height=chunk_length_pixels)

    file_info = f'{tile_id}__{bounds_str}'

    s3_client = boto3.client("s3")

    dask_print(f"Saving {bounds_str} in {tile_id} for {year}: {timestr()}")

    # For every output file, saves from array to local raster, then to s3.
    # Can't save directly to s3, unfortunately, so need to save locally first.
    for key, value in output_dict.items():

        file_name = f"{file_info}__{key}__{timestr()}"

        with rasterio.open(f"/tmp/{file_name}.tif", 'w', driver='GTiff', width=chunk_length_pixels, height=chunk_length_pixels, count=1, dtype='uint8', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst:
            dst.write(value[0].astype(rasterio.uint8), 1)

        dask_print(f"Uploading {bounds_str} in {tile_id} for {year}: {timestr()}")

        s3_client.upload_file(f"/tmp/{file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{value[2]}/{file_name}.tif")

        # Deletes the local raster. It won't be used again.
        os.remove(f"/tmp/{file_name}.tif")

<font size="4">Model steps</font>

In [9]:
# Function to reclassify GLCLU classes to basic IPCC reporting classes.
# Operates on the array/chunk. 
# Classification comes from https://onewri-my.sharepoint.com/:p:/g/personal/david_gibbs_wri_org/EWwyxRfgdeVJi4ezwX7LrfcBjCoqAcjL2jRAZjb_8RU9LQ?e=YUsQiU
def reclassify_to_IPCC(GLCLU_block):

    # Outputs
    IPCC_classes = np.zeros(GLCLU_block.shape)

    IPCC_classes[np.where(GLCLU_block <= 1)] = otherland                                 
    IPCC_classes[np.where((GLCLU_block >= 2) & (GLCLU_block <= 26))] = grassland          
    IPCC_classes[np.where((GLCLU_block >= 27) & (GLCLU_block <= 48))] = forest         
    IPCC_classes[np.where((GLCLU_block >= 100) & (GLCLU_block <= 101))] = wetland       
    IPCC_classes[np.where((GLCLU_block >= 102) & (GLCLU_block <= 126))] = grassland       
    IPCC_classes[np.where((GLCLU_block >= 127) & (GLCLU_block <= 148))] = forest       
    IPCC_classes[np.where((GLCLU_block >= 200) & (GLCLU_block <= 204))] = wetland       
    IPCC_classes[np.where((GLCLU_block >= 205) & (GLCLU_block <= 207))] = otherland       
    IPCC_classes[np.where(GLCLU_block == 241)] = otherland                                
    IPCC_classes[np.where(GLCLU_block == 244)] = cropland                                
    IPCC_classes[np.where(GLCLU_block == 250)] = settlement                               
    IPCC_classes[np.where(GLCLU_block == 254)] = otherland                              
    
    return IPCC_classes

In [10]:
# Function to map basic IPCC change classes.
# Operates pixel by pixel, so uses numba (Python compiled to C++).
@jit(nopython=True)
def change_classes_IPCC(IPCC_previous_block, IPCC_current_block):

    # Output array of 0s
    IPCC_change_block = np.zeros(IPCC_previous_block.shape)

    # Iterates through all pixels in the chunk
    for row in range(IPCC_previous_block.shape[0]):
        for col in range(IPCC_previous_block.shape[1]):

            IPCC_previous = IPCC_previous_block[row, col]
            IPCC_current = IPCC_current_block[row, col]

            # Equation to calculate the IPCC change code
            IPCC_change_block[row, col] = ((IPCC_previous - 1) * IPCC_class_max_val) + IPCC_current

    return IPCC_change_block

In [11]:
# Downloads input chunks, reclassifies GLCLU classes into IPCC land use reporting classes for each year, and maps changes between classes for consecutive years.
# Chunks are defined by a bounding box and a starting year for iteration
def reclassify_and_map_change_chunk(bounds, start_year):

    bounds_str = boundstr(bounds)    # String form of chunk bounds
    tile_id = xy_to_tile_id(bounds[0], bounds[3])    # tile_id in YYN/S_XXXE/W
    chunk_length_pixels = calc_chunk_length_pixels(bounds)   # Chunk length in pixels (as opposed to decimal degrees)

    
    ### Part 1: download tiles

    # Dictionary of downloaded layers
    layers = {}

    download_dict = {
        
        # "iso": f"s3://gfw2-data/gadm_administrative_boundaries/v3.6/raster/epsg-4326/10/40000/adm0/gdal-geotiff/{tile_id}.tif",  # Originally from gfw-data-lake, so it's in 400x400 windows,

        "land_cover_2000": f"s3://gfw2-data/landcover/composite/2000/raw/{tile_id}.tif",
        "land_cover_2005": f"s3://gfw2-data/landcover/composite/2005/raw/{tile_id}.tif",
        "land_cover_2010": f"s3://gfw2-data/landcover/composite/2010/raw/{tile_id}.tif",
        "land_cover_2015": f"s3://gfw2-data/landcover/composite/2015/raw/{tile_id}.tif",
        "land_cover_2020": f"s3://gfw2-data/landcover/composite/2020/raw/{tile_id}.tif"   
    }
    
    futures = prepare_to_download_chunk(bounds, start_year, download_dict)
    dask_print(f"Waiting for requests for data in chunk {bounds_str} in {tile_id}: {timestr()}")
    # Waits for requests to come back with data from S3
    
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    
    ### Part 2: reclassify GLCLU classes into IPCC reporting classes 
    IPCC_class_dict = {}

    # Iterates through model years
    for year in list(range(2000, 2021, 5)):
        
        dask_print(f"Reclassifying {bounds_str} in {tile_id} for {year}: {timestr()}")

        # Reclassifies GLCLU to 6 IPCC classes 
        IPCC_classes = reclassify_to_IPCC(
            layers[f"land_cover_{year}"]   
        )

        # Output files to upload to s3
        IPCC_class_dict[f"IPCC_classes_{year}"] = [IPCC_classes, "uint8", "IPCC_basic_classes"]                 

        # Clear memory of unneeded arrays
        del IPCC_classes

    save_and_upload(bounds, chunk_length_pixels, tile_id, bounds_str, year, IPCC_class_dict)

    
    ### Part 3
    IPCC_change_dict = {}

    # Iterates through model years in a way that change can be calculated
    for year in list(range(2005, 2021, 5)):
        
        dask_print(f"Getting IPCC class change in {bounds_str} in {tile_id} for {year}: {timestr()}")

        # Maps change between IPCC classes
        IPCC_change = change_classes_IPCC(
            IPCC_class_dict[f"IPCC_classes_{year-5}"][0], # first [0] needed because results_download is a tuple with a dictionary inside it. Second [0] to isolate the array.
            IPCC_class_dict[f"IPCC_classes_{year}"][0]    # first [0] needed because results_download is a tuple with a dictionary inside it. Second [0] to isolate the array.
        )

        # Output files to upload to s3
        IPCC_change_dict[f"IPCC_change_{year-5}_{year}"] = [IPCC_change, "uint8", "IPCC_basic_change"]  

        # Clear memory of unneeded arrays
        del IPCC_change

    save_and_upload(bounds, chunk_length_pixels, tile_id, bounds_str, year, IPCC_change_dict)

    return f"success for {bounds_str}: {timestr()}"

In [12]:
%%time

# Year to start the analysis
# start_year = 2000   # full run
start_year = 2020  # final year

# Area to analyze
# chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-12, 34, 32, 72, 1]  # all of Europe
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks
# chunk_params = [10, 40, 20, 50, 1]    # 10x10 deg (50N_010E), 100 chunks
chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no data)

# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(reclassify_and_map_change_chunk)(chunk, start_year) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

Processing 1 chunks
Making requests for data in chunk 10_40_20_50 in 50N_010E: 20240124_22_07_31
Waiting for requests for data in chunk 10_40_20_50 in 50N_010E: 20240124_22_09_17
Reclassifying 10_40_20_50 in 50N_010E for 2000: 20240124_22_09_17
Reclassifying 10_40_20_50 in 50N_010E for 2005: 20240124_22_10_46
Reclassifying 10_40_20_50 in 50N_010E for 2010: 20240124_22_12_15
Reclassifying 10_40_20_50 in 50N_010E for 2015: 20240124_22_13_44
Making requests for data in chunk 10_40_20_50 in 50N_010E: 20240124_22_14_10


KeyboardInterrupt: 

In [ ]:
# To run without dask at all
process_chunk([10, 49, 11, 50], 1, start_year)

In [ ]:
# Download test-- checks that uri is found and recognized
tile_id = "50N_010E"
# uri = f"s3://gfw2-data/climate/carbon_model/BGB_AGB_ratio/processed/20230216/{tile_id}_BGB_AGB_ratio.tif"
uri = f"s3://gfw2-data/gadm_administrative_boundaries/v3.6/raster/epsg-4326/10/40000/adm0/gdal-geotiff/{tile_id}.tif"  # Originally from gfw-data-lake, so it's in 400x400 windows
# uri = f"s3://gfw2-data/fao_ecozones/v2000/raster/epsg-4326/10/40000/class/gdal-geotiff/{tile_id}.tif"   # Originally from gfw-data-lake, so it's in 400x400 windows 
bounds = [10, 49.75, 10.25, 50]

get_tile_dataset_rio(uri, bounds, 1000)

In [ ]:
coiled_client.restart() 

In [ ]:
client.cancel(future) # per https://github.com/dask/distributed/issues/3898#issuecomment-645590511

In [ ]:
# aws s3 cp s3://gfw2-data/climate/European_height_carbon_model/outputs/ . --recursive --exclude "*" --include "*10_49_11_50*"
# aws s3 cp s3://gfw2-data/climate/European_height_carbon_model/outputs/ . --recursive --exclude "*" --include "*2002*10_49_11_50*"